# Project 2.3B

<img src="Data/movietables.png">

In [66]:
import pandas as pd
import json
import numpy as np
import pymysql
from sqlalchemy import create_engine

In [9]:
with open("/Users/rowenahan/.secret/tmdb_sql.json") as f:
    code=json.load(f)
print(code.keys())

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

## Download data

In [88]:
basics=pd.read_csv('Data/title_basics_cleaned.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [89]:
basics.columns

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')

In [90]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86979 entries, 0 to 86978
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86979 non-null  object 
 3   originalTitle   86979 non-null  object 
 4   isAdult         86979 non-null  int64  
 5   startYear       86979 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  86979 non-null  int64  
 8   genres          86979 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 6.0+ MB


### "Genres" columns

In [91]:
type(basics["genres"])

pandas.core.series.Series

In [30]:
basics["genres"]=basics["genres"].apply(json.loads)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [92]:
#Slicing out a row for testing
test_coord=basics.loc[5,'genres']
test_coord

'Comedy,Drama,Fantasy'

In [93]:
#What type is the column
type(test_coord)

str

In [94]:
#testing str.split with expand=True
basics["genres"].str.split(",", expand=True)

,0,1,2
0,Comedy,Fantasy,Romance
1,Drama,None,None
2,Drama,None,None
3,Comedy,Horror,Sci-Fi
4,Drama,None,None
...,...,...,...
86974,Drama,None,None
86975,Comedy,Drama,Fantasy
86976,Drama,None,None
86977,Action,Adventure,Thriller


In [95]:
#testing .str.split wiht out expand=True
basics["genres"].str.split(",")

0           [Comedy, Fantasy, Romance]
1                              [Drama]
2                              [Drama]
3             [Comedy, Horror, Sci-Fi]
4                              [Drama]
                     ...              
86974                          [Drama]
86975         [Comedy, Drama, Fantasy]
86976                          [Drama]
86977    [Action, Adventure, Thriller]
86978                 [Drama, History]
Name: genres, Length: 86979, dtype: object

In [96]:
#Looing at values of genres
basics["genres"].value_counts()

Drama                        17085
Comedy                        7148
Horror                        4071
Comedy,Drama                  4000
Drama,Romance                 2623
                             ...  
Music,Mystery,Romance            1
History,Horror,Mystery           1
Crime,Music,Mystery              1
Crime,Fantasy,Romance            1
Biography,Fantasy,Musical        1
Name: genres, Length: 854, dtype: int64

In [97]:
#making new column ('genres_split'), with .str.split 
basics["genres_split"]=basics["genres"].str.split(',')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama,[Drama]
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,[Drama]
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama,[Drama]


In [98]:
#using .explode on "genres_split" column
explode=basics.explode("genres_split")
explode[["primaryTitle","genres","genres_split"]]

,primaryTitle,genres,genres_split
0,Kate & Leopold,"Comedy,Fantasy,Romance",Comedy
0,Kate & Leopold,"Comedy,Fantasy,Romance",Fantasy
0,Kate & Leopold,"Comedy,Fantasy,Romance",Romance
1,The Tango of the Widower and Its Distorting Mi...,Drama,Drama
2,The Other Side of the Wind,Drama,Drama
...,...,...,...
86977,Safeguard,"Action,Adventure,Thriller",Action
86977,Safeguard,"Action,Adventure,Thriller",Adventure
86977,Safeguard,"Action,Adventure,Thriller",Thriller
86978,Coven,"Drama,History",Drama


In [101]:
# WAS ON DIFFEREN LESSON MAY HAVE DONE OUT OF TURN
unique_genres=sorted(explode["genres_split"].unique())
unique_genres

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

In [103]:
#creating integers for each id
int_ids = range(len(unique_genres))
int_ids

range(0, 25)

In [104]:
# Converting our range to a list and showing the first 10 values
example_range=list(int_ids)
example_range[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [105]:
#Zip together the unique_genres as the keys and the int_ids as the value
id_map=dict(zip(unique_genres,int_ids))
id_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Sport': 20,
 'Talk-Show': 21,
 'Thriller': 22,
 'War': 23,
 'Western': 24}

In [108]:
#testing 
example_str_id = 'Sci-Fi'
id_map[example_str_id]

19

In [109]:
#matching the id_map to "genres_split"
explode["genres_split"].map(id_map)

0         5
0         9
0        18
1         7
2         7
         ..
86977     0
86977     2
86977    22
86978     7
86978    11
Name: genres_split, Length: 162600, dtype: int64

In [111]:
#overwriting the original "genres_split" column with id_map
explode["genres_split"]=explode["genres_split"].replace(id_map)
explode.head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",5
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",9
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",18
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama,7
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,7
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi",5
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi",12
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi",19
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama,7
5,tt0100275,movie,The Wandering Soap Opera,La Telenovela Errante,0,2017.0,NaN,80,"Comedy,Drama,Fantasy",5


In [112]:
#Saving just tconst and genres_split as new df
title_genres=explode[['tconst', 'genres_split']].copy()
title_genres.head()

,tconst,genres_split
0,tt0035423,5
0,tt0035423,9
0,tt0035423,18
1,tt0062336,7
2,tt0069049,7


In [114]:
#Saving dict as new dataframe
genre_lookup=pd.DataFrame({"genre_name": id_map.keys(), "genre_id": id_map.values()})
genre_lookup.head()

,genre_name,genre_id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


In [118]:
# drop genres columns
explode=explode.drop(columns=["genres", 'genres_split'])
#save data tor next lesson
explode.to_csv('Data/advance_tf_data_pt1.csv', index=False)